In [25]:
import openai
import os
import pandas as pd
from langchain.llms import OpenAI
import IPython
from sentence_transformers import SentenceTransformer
import numpy as np

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

import re
from langchain.document_loaders.parsers import BS4HTMLParser, PDFMinerParser
from langchain.document_loaders.parsers.generic import MimeTypeBasedParser
from langchain.document_loaders.parsers.txt import TextParser
import requests
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import arxiv
import PyPDF2
import io
from operator import itemgetter
#from langchain.chains import RetrievalQA
#from langchain_community.document_loaders import BSHTMLLoader
#from langchain_community.document_loaders import Blob
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from pycm import ConfusionMatrix
from sklearn.metrics import classification_report

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Function to convert PDF to text
def pdf_to_txt(input_file):

    with open(input_file, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ''

        # Extract text from each page of the PDF
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    
    return text

def prepare_index(output_file):
     loader = TextLoader(output_file)
     index = VectorstoreIndexCreator().from_loaders([loader])
     return index


## Building Evaluation framework to evaluate above responses.
 For Evaluation, my aim is to generate responses from PaperClip generating answers to signle questions : " What is the title of the paper?"".I am manually generating the ground truth of 11 papers that are stored in the folder ;  "PaperDB". 

 Output -  I am generating a dataframe with quetsion, answer of LLM ( paper name), relevant retrived context. I compliment that with groundtruth

In [6]:
### Iterating and genrating response for 11 papers

folder_path = "/Users/apoorvaacharya/Documents/GitHub/maven-pe-for-llms-8/PaperClip/PaperDB"
txt_folder_path = "/Users/apoorvaacharya/Documents/GitHub/maven-pe-for-llms-8/PaperClip/PaperDB_txt"
query = "What is the title of this paper?"

papers = []
contexts = []
answers = []
questions = []

for file in os.listdir(folder_path):
    
    if file.endswith('.pdf'):
        print(file)
        paper_pdf_path = os.path.join(folder_path, file)
        mytxt = pdf_to_txt(paper_pdf_path)
        paper_txt_filename = os.path.join(folder_path, os.path.splitext(file)[0] + '.txt')
        
        with io.open(paper_txt_filename, 'w', encoding='utf-8') as f:
            f.write(mytxt)

        index_paper = prepare_index(paper_txt_filename)

        text_splitter_paper = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap =  50) #CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
        chunks_paper = text_splitter_paper.split_text(mytxt)
        #print("mytxt", mytxt)
        embeddings_paper = OpenAIEmbeddings()
        ## creating vector store

        docsearch_paper = Chroma.from_texts(chunks_paper, embeddings_paper, metadatas=[{"source": str(i)} for i in range(len(chunks_paper))])
        # rag retrieval based on similarity search
        docs_i = docsearch_paper.similarity_search(query)
        #print("docs_i", docs_i[0].page_content)
        ids = [str(i) for i in range(1, len(os.listdir(folder_path)) + 1)]

        #docs_i = [item[0] for item in docs_all]
        # print("count after", docsearch_paper._collection.count())

        # modelling response generation with prompting to return only based on the context retrived 
        retriever_paper = docsearch_paper.as_retriever()
        #prompt = hub.pull("rlm/rag-prompt")

        template = """You are a personal learning agent. 
        Output "NA" if the question is irrevant 
        Otherwise, Answer the question based only on the following context: {context} Only output the title

        Question: {question}

        """
        prompt = ChatPromptTemplate.from_template(template)

        chain = (
            {
                "context": itemgetter("question") | retriever_paper,
                "question": itemgetter("question")
            }
            | prompt
            | OpenAI(temperature=0.3)
            | StrOutputParser()
        )

        answer2 = chain.invoke({"question": query, "context": itemgetter("question") | retriever_paper})

       
        print("Question:", query)
        print("Answer:",answer2)
        #print(docs_i)
        papers.append(file)
        contexts.append(np.asarray(list(map(lambda doci: [doci.page_content], docs_i))))
        answers.append(answer2)
        questions.append(query)
        #print("count before", docsearch_paper._collection.count())
        docsearch_paper.delete_collection()

# answer_withprompt = output['output_text']
print(contexts)

eval_df = pd.DataFrame({"documents": papers, "contexts": contexts, "answer":answers, "question": questions })
eval_csv_name = os.path.join(folder_path, 'PaperClip_eval_dataset_raw.csv')
eval_df.to_csv(eval_csv_name)

2303.18223.pdf


/Users/apoorvaacharya/miniforge3/envs/pe-for-llms/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/apoorvaacharya/miniforge3/envs/pe-for-llms/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_dep

Question: What is the title of this paper?
Answer: 
Attention is all you need
2403.08770.pdf


unknown widths : 
[0, IndirectObject(462, 0, 11014247184)]


Question: What is the title of this paper?
Answer: 
NA
2306.07745.pdf
Question: What is the title of this paper?
Answer: 
Discovering faster matrix multiplication algorithms with reinforcement learning.
9110037.pdf
Question: What is the title of this paper?
Answer: 
The title of this paper is "Inflationary Cosmology in the Light of the Latest Observations".
0911.0652.pdf
Question: What is the title of this paper?
Answer: 
Answer: Brainstorming through the Sequence Universe: Theories on the Protein Problem
2403.09580.pdf
Question: What is the title of this paper?
Answer: 
Causality: Models, Reasoning and Inference
2403.07769.pdf
Question: What is the title of this paper?
Answer: 
The Archetypes and the Collective Unconscious.
9502007.pdf
Question: What is the title of this paper?
Answer: 
NA
2403.07794.pdf
Question: What is the title of this paper?
Answer: 
The 2018 Conference on Empirical Methods in Natural Language Processing
2311.14682.pdf
Question: What is the title of this paper?
A

In [7]:
eval_df

,documents,contexts,answer,question
0,2303.18223.pdf,[[sociation for Computational Linguistics: Hum...,\nAttention is all you need,What is the title of this paper?
1,2403.08770.pdf,[[problem: recent algorithmic advances . Sprin...,\nNA,What is the title of this paper?
2,2306.07745.pdf,[[problem whether the suboptimal regret bounds...,\nDiscovering faster matrix multiplication alg...,What is the title of this paper?
3,9110037.pdf,"[[[35] T.S. Bunch and P.C.W. Davies, Proc. R. ...","\nThe title of this paper is ""Inflationary Cos...",What is the title of this paper?
4,0911.0652.pdf,"[[experimentally validated ontology. Further,...",\nAnswer: Brainstorming through the Sequence U...,What is the title of this paper?
5,2403.09580.pdf,"[[J. Pearl. Causality: Models, Reasoning and I...","\nCausality: Models, Reasoning and Inference",What is the title of this paper?
6,2403.07769.pdf,[[https://www.researchgate.net/publication/284...,\nThe Archetypes and the Collective Unconscious.,What is the title of this paper?
7,9502007.pdf,"[[puters, NPAC technical report SCCS-279 (1992...",\nNA,What is the title of this paper?
8,2403.07794.pdf,[[the 2018 Conference on Empirical Methods in ...,\nThe 2018 Conference on Empirical Methods in ...,What is the title of this paper?
9,2311.14682.pdf,"[[written on this topic, some of these were co...",\nData-Driven Models for studying the Dynamics...,What is the title of this paper?


# Evaluation Technique 1: RAGAS: Automated Evaluation of Retrieval Augmented Generation

 I'm taking responses from Step 2: Model with basic instruction prompt and evaluating the LLM model using the [ragas](https://github.com/explodinggradients/ragas) evaluation tool. Uses `gpt-3.5` ad default model for evaluation.

In [40]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
from ast import literal_eval
import matplotlib.pyplot as plt

eval_df_groundtruth = pd.read_csv("/Users/apoorvaacharya/Documents/GitHub/maven-pe-for-llms-8/PaperClip/PaperClip_eval_dataset_wGT.csv")

print(eval_df_groundtruth.columns)
eval_df_groundtruth.rename(columns = {"ground_truth": "ground_truths"}, inplace=True)
eval_df_groundtruth['ground_truths'] = eval_df_groundtruth['ground_truths'].apply(lambda row: [row])
eval_df_groundtruth['contexts'] = eval_df_groundtruth['contexts'].apply(lambda row: literal_eval(row))
eval_df_groundtruth.drop(['Unnamed: 0', 'documents'], axis=1, inplace=True)
dataset = ds.dataset(pa.Table.from_pandas(eval_df_groundtruth).to_batches())
eval_arrow_dataset = Dataset(pa.Table.from_pandas(eval_df_groundtruth))
eval_arrow_dataset

Index(['Unnamed: 0', 'documents', 'contexts', 'answer', 'ground_truth',
       'question', 'relevant'],
      dtype='object')


Dataset({
    features: ['contexts', 'answer', 'ground_truths', 'question', 'relevant'],
    num_rows: 17
})

In [5]:
eval_arrow_dataset[0]

{'contexts': ['this survey, and improve the quality as much as we can.\nFor us, survey writing is also a learning process for LLMs\nby ourselves. For readers with constructive suggestions to\nimprove this survey, you are welcome to leave comments on\nthe GitHub page of our survey or directly email our authors.\nWe will make revisions following the received comments\nor suggestions in a future version, and acknowledge the\nreaders who have contributed constructive suggestions in\nour survey.\nUpdate log . In this part, we regularly maintain an update\nlog for the submissions of this survey to arXiv:\n•First release on March 31, 2023: the initial version.\n•Update on April 9, 2023: add the affiliation information,\nrevise Figure 3 and Table 1 and clarify the correspond-\ning selection criterion for LLMs, improve the writing,\nand correct some minor errors.\n•Update on April 11, 2023: correct the errors for library\nresources.\n•Update on April 12, 2023: revise Figure 3 and Table 1,\nand 

In [6]:
# the selected metrics
import ragas

from ragas.metrics import (
    context_precision,
    faithfulness,
    answer_relevancy,
    context_recall,
)

from ragas import evaluate

result = evaluate(
    eval_arrow_dataset, # selecting only 3
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating: 100%|██████████| 68/68 [01:34<00:00,  1.39s/it]


{'context_precision': 0.3529, 'faithfulness': 0.6667, 'answer_relevancy': 0.8440, 'context_recall': 0.7647}

In [9]:
df = result.to_pandas()
df

,contexts,answer,ground_truths,question,relevant,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,"[this survey, and improve the quality as much ...","The title of this paper is ""LLM Survey: A Comp...",[Parallel Cluster Labeling for Large-Scale Mon...,What is the title of this paper?,No,Parallel Cluster Labeling for Large-Scale Mont...,0.0,0.0,1.000000,0.0
1,[problem: recent algorithmic advances . Spring...,Deep Global Registration,[A Survey of Large Language Models],What is the title of this paper?,No,A Survey of Large Language Models,0.0,1.0,0.787506,0.0
2,[problem whether the suboptimal regret bounds ...,Discovering faster matrix multiplication algor...,[FastMAC: Stochastic Spectral Sampling of Corr...,What is the title of this paper?,No,FastMAC: Stochastic Spectral Sampling of Corre...,0.0,0.0,0.770152,1.0
3,"[[35] T.S. Bunch and P.C.W. Davies, Proc. R. S...","The title of this paper is ""Inflation and Quan...",[Kernelized Reinforcement Learning with Order ...,What is the title of this paper?,No,Kernelized Reinforcement Learning with Order O...,0.0,NaN,1.000000,1.0
4,"[experimentally validated ontology. Further, ...",\nAnswer: Brainstorming through the Sequence U...,[HARD ART OF THE UNIVERSE CREATION (Stochastic...,What is the title of this paper?,No,HARD ART OF THE UNIVERSE CREATION (Stochastic ...,0.0,1.0,0.934616,1.0
5,"[J. Pearl. Causality: Models, Reasoning and In...","\nCausality: Models, Reasoning and Inference",[Brainstorming through the Sequence Universe:\...,What is the title of this paper?,No,Brainstorming through the Sequence Universe:\n...,0.0,NaN,0.839927,1.0
6,[https://www.researchgate.net/publication/2844...,The Kimball Group Reader,[Algorithmic syntactic causal identiﬁcation],What is the title of this paper?,No,Algorithmic syntactic causal identiﬁcation,0.0,0.0,0.899254,0.0
7,"[puters, NPAC technical report SCCS-279 (1992)...",None,[Transforming Competition into Collaboration: ...,What is the title of this paper?,No,Transforming Competition into Collaboration: T...,0.0,NaN,0.000000,0.0
8,[the 2018 Conference on Empirical Methods in N...,None,[Fine-Tuning Large Language Models with Sequen...,What is the title of this paper?,No,Fine-Tuning Large Language Models with Sequent...,1.0,NaN,0.829730,1.0
9,"[written on this topic, some of these were com...",Data-Driven Models for studying the Dynamics o...,[Data-Driven Models for studying the Dynamics ...,What is the title of this paper?,Yes,Data-Driven Models for studying the Dynamics o...,1.0,1.0,0.925336,1.0


## Results:

### 'context_precision': 0.3529,
### 'faithfulness': 0.6364,
### 'answer_relevancy': 0.8347,
### 'context_recall': 0.6471

# Evaluation Technique 2: RAG using Phoenix 


In [10]:
# !pip install 'arize-phoenix[evals]' ipython matplotlib openai pycm scikit-learn
!pip install -qq "arize-phoenix-evals" "openai>=1" ipython matplotlib pycm scikit-learn tiktoken nest_asyncio


In [42]:
from phoenix.evals import (
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    OpenAIModel,
    download_benchmark_dataset,
    llm_classify,
)
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
from pycm import ConfusionMatrix


df = eval_df_groundtruth.rename(
    columns={
        "question": "input",
        "contexts": "reference",
    },
)
model = OpenAIModel(
    model="gpt-4",
    temperature=0.0,
)
rails =list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values())
df[["eval_relevance"]] = llm_classify(df, model, RAG_RELEVANCY_PROMPT_TEMPLATE, rails)
#Golden dataset has True/False map to -> "irrelevant" / "relevant"
#we can then scikit compare to output of template - same format
y_true = df["relevant"].map({"Yes": "relevant", "No": "irrelevant"})
y_pred = df["eval_relevance"]

# Compute Per-Class Precision, Recall, F1 Score, Support
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

🐌!! If running llm_classify inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
llm_classify |██████████| 17/17 (100.0%) | ⏳ 01:20<00:00 |  4.71s/it
/Users/apoorvaacharya/miniforge3/envs/pe-for-llms/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/apoorvaacharya/miniforge3/envs/pe-for-llms/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
df

,reference,answer,ground_truths,input,relevant,eval_relevance
0,"[this survey, and improve the quality as much ...","The title of this paper is ""LLM Survey: A Comp...",[Parallel Cluster Labeling for Large-Scale Mon...,What is the title of this paper?,No,unrelated
1,[problem: recent algorithmic advances . Spring...,Deep Global Registration,[A Survey of Large Language Models],What is the title of this paper?,No,unrelated
2,[problem whether the suboptimal regret bounds ...,Discovering faster matrix multiplication algor...,[FastMAC: Stochastic Spectral Sampling of Corr...,What is the title of this paper?,No,unrelated
3,"[[35] T.S. Bunch and P.C.W. Davies, Proc. R. S...","The title of this paper is ""Inflation and Quan...",[Kernelized Reinforcement Learning with Order ...,What is the title of this paper?,No,unrelated
4,"[experimentally validated ontology. Further, ...",\nAnswer: Brainstorming through the Sequence U...,[HARD ART OF THE UNIVERSE CREATION (Stochastic...,What is the title of this paper?,No,relevant
5,"[J. Pearl. Causality: Models, Reasoning and In...","\nCausality: Models, Reasoning and Inference",[Brainstorming through the Sequence Universe:\...,What is the title of this paper?,No,unrelated
6,[https://www.researchgate.net/publication/2844...,The Kimball Group Reader,[Algorithmic syntactic causal identiﬁcation],What is the title of this paper?,No,unrelated
7,"[puters, NPAC technical report SCCS-279 (1992)...",NaN,[Transforming Competition into Collaboration: ...,What is the title of this paper?,No,unrelated
8,[the 2018 Conference on Empirical Methods in N...,NaN,[Fine-Tuning Large Language Models with Sequen...,What is the title of this paper?,No,unrelated
9,"[written on this topic, some of these were com...",Data-Driven Models for studying the Dynamics o...,[Data-Driven Models for studying the Dynamics ...,What is the title of this paper?,Yes,relevant


In [43]:
print(" precision, recall, f1 score :", precision[1], recall[1], round((f1[1]),3))

print(classification_report(y_true, y_pred))


 precision, recall, f1 score : 0.6666666666666666 0.4 0.5
              precision    recall  f1-score   support

  irrelevant       0.00      0.00      0.00        12
    relevant       0.67      0.40      0.50         5
   unrelated       0.00      0.00      0.00         0

    accuracy                           0.12        17
   macro avg       0.22      0.13      0.17        17
weighted avg       0.20      0.12      0.15        17



/Users/apoorvaacharya/miniforge3/envs/pe-for-llms/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/apoorvaacharya/miniforge3/envs/pe-for-llms/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/apoorvaacharya/miniforge3/envs/pe-for-llms/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [38]:
df.to_csv("/Users/apoorvaacharya/Documents/GitHub/maven-pe-for-llms-8/PaperClip/eval_phoenix.csv")

### Findings:
context precision by ragas was ~ 35% whie phoenix rag evals gave me 67%.
Although, the phoenix evals are more precise sine it required ground truth at the rag/context level. 
Ease of use is great and comparable.


